<a href="https://colab.research.google.com/github/InryeolChoi/analysis_basic/blob/main/1%EC%9E%A5_%26_2%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-4. 구글 드라이브와 연동


In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [10]:
with open("/gdrive/My Drive/test.txt", 'w') as f:
  f.writelines("test")

# 2-3. 어휘집합 구축하기

In [ ]:
!pip install ratsnlp

말뭉치 내려받기 및 전처리

In [5]:
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 113MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 81.5MB/s]


NSMC 전처리

In [7]:
import os
def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')
write_lines("/content/train.txt", nsmc.train.get_all_texts())
write_lines("/content/test.txt", nsmc.train.get_all_texts())

디렉터리 만들기

In [13]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok = True)

바이트 수준 BPE 어휘 집합 구축

In [14]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenzier = ByteLevelBPETokenizer()
bytebpe_tokenzier.train(
    files = ["/content/train.txt", "/content/test.txt"],
    vocab_size = 10000,
    special_tokens = ["[PAD]"]
)
bytebpe_tokenzier.save_model("/gdrive/My Drive/nlpbook/bbpe")

['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

In [17]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

In [19]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files = ["/content/train.txt", "/content/test.txt"],
    vocab_size = 10000
)
wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

# 2-4. 토큰화하기

준비코드




In [ ]:
!pip install ratsnlp

In [21]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

GPT 입력값 설정

In [28]:
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /gdrive/My Drive/nlpbook/bbpe/config.json not found


In [29]:
Sentences = [
    "아 더빙...진짜 짜증나네요 목소리",
    "연기가 별로 답이 없는 것 같다.",
    "별루였다..."
]
tokenized_sentences = [tokenizer_gpt.tokenize(sen) for sen in Sentences]
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '...', 'ì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['ìĹ°ê¸°ê°Ģ', 'Ġë³Ħë¡ľ', 'ĠëĭµìĿ´', 'ĠìĹĨëĬĶ', 'Ġê²ĥ', 'Ġê°Ļëĭ¤', '.'],
 ['ë³Ħë£¨', 'ìĺĢëĭ¤', '...']]

In [31]:
batch_inputs = tokenizer_gpt(
    Sentences,
    padding = "max_length",
    max_length = 12,
    truncation = True
)

BERT 입력값 만들기

In [37]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case = False
)

file /gdrive/My Drive/nlpbook/wordpiece/config.json not found


In [39]:
Sentences = [
    "아 더빙...진짜 짜증나네요 목소리",
    "연기가 별로 답이 없는 것 같다.",
    "별루였다..."
]
tokenized_sentences = [tokenizer_bert.tokenize(sen) for sen in Sentences]
tokenized_sentences

[['아', '더빙', '.', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['연기가', '별로', '답이', '없는', '것', '같다', '.'],
 ['별루', '##였다', '.', '.', '.']]

In [41]:
batch_inputs = tokenizer_bert(
    Sentences,
    padding = "max_length",
    max_length = 12,
    truncation = True
)
